In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

C:\Users\cep4u\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\cep4u\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\cep4u\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# Define the TF-IDF vectorizer with desired parameters
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: [word_pos[0] for word_pos in eval(x)],lowercase=False,stop_words=None)

def tfidf_vectorize(data, column):
    # Fit and transform the specified column
    tfidf_vectors = tfidf_vectorizer.fit_transform(data[column])

    # Return the resulting sparse matrix
    return tfidf_vectors

In [3]:
def train_model(X, y):
    # Initialize a KFold object with 5 folds
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Create lists to store the training history and trained models for each fold
    models = []
    
    # Iterate over the folds and train the model on each one
    for fold, (train_indices, val_indices) in enumerate(kfold.split(X, y)):
        # Split the data into training and validation sets for this fold
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        # Create a svm model
        model = LogisticRegression(random_state=42, multi_class="ovr")

          # Train the model on the input data
        model.fit(X, y)

        # Add the trained model and training history to the lists
        models.append(model)

    # Return the list of trained models and training histories
    return models

In [4]:
df = pd.read_csv("C:\\Users\\cep4u\\JHU Classes\\Classes\\Spring 2023\\Theory of Machine Learning\\Group Project\\625.742-Theory-of-Machine-Learning\\News Category Dataset Cleaned 20230416.csv")

In [5]:
categories = ['WELLNESS & HEALTH','ENTERTAINMENT','TRAVEL', 'STYLE & BEAUTY', 'PARENTING', 'QUEER VOICES', 'FOOD & DRINK']
data = df[df['category'].isin(categories)]

#sample equal subsets of 7 categories
sampled_data = data.groupby('category', group_keys=False).apply(lambda x: x.sample(n=3000))

# Encode the target variable
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sampled_data['category'])

# Vectorize the text data using TF-IDF
X_sparse = tfidf_vectorize(sampled_data, 'wordnet_pos_hdl')

In [6]:
# Withold validation set.
X_train_sparse, X_val_sparse, y_train, y_val = train_test_split(X_sparse, y, test_size=0.2, random_state=12)

# Convert the sparse matrices to dense arrays
X = X_train_sparse.toarray()

In [7]:
# Train the support vector machine on the training data
models = train_model(X, y_train)

In [8]:
acc_avg = []
X_val = X_val_sparse.toarray()
for model in models: # Evaluate the model on the testing data
    accuracy = model.score(X_val, y_val)
    acc_avg.append(accuracy)
    
print('Test accuracy:', sum(acc_avg)/len(acc_avg))

Test accuracy: 0.7604761904761905


In [9]:
acc_avg = []
X_val = X_val_sparse.toarray()
for model in models: # Evaluate the model on the testing data
    accuracy = model.score(X, y_train)
    acc_avg.append(accuracy)
    
print('Test accuracy:', sum(acc_avg)/len(acc_avg))

Test accuracy: 0.8972619047619048
